In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pickle # 1, 쿠키값을 저장하기 위한 라이브러리 pickle
import time # 2, 최초 1회 인증받는데 필요한시간


options = Options()
options.add_argument("--start-maximized") # 3, 현재 우리 브라우저에 최적화된 사이즈로 크롤링하게끔하는 옵션
options.add_argument("window-size=1920x1080") 
options.add_argument("lang=ko_KR")

service = Service(ChromeDriverManager().install()) # 4, 크롬드라이브매니저를 우리의 인풋에 맞게 설정하고 시작하겠다!

driver = webdriver.Chrome(service=service, options=options) # 5,서비스와 옵션의 두가지 값을 드라이버에 옵션값으로 설정! 드라이버가 크롤링

driver.get("https://www.instagram.com/accounts/login/#") # 6, 인스타그램의 로그인페이지를 찾아와

# 셀레니움의 동적인 페이지를 찾아오듯이 로그인하면 안됨. 수동으로 로그인 해야함.
#아이디와 패스워드
print("수동으로 인증시작! in 30.sec")
time.sleep(30)

# 쿠키값을 가져와서 저장하기 instagram이 주는 쿠키의 형태가 있음 "instagram_cookies.pkl"
# 이 이름으로 쿠키를 저장해서 써야함. wb = "write Binary"
with open("instagram_cookies.pkl","wb") as f : # 7, 이 이름으로 저장한 후에 저장되어진 요소를 열어야함 = 이 행위를 f 라는 이름을씀
    pickle.dump(driver.get_cookies(),f) # 8, 드라이버가 사이트를 찾아올때 내려주는 쿠키값을 싹 가져와서 저 이름으로 쓰고 그 값을 쓰는 명령어
    # 9, 이러한 순서와 절차를 알고리즘! dump는 직렬형태로 찾아와서 특정 쿠키값을 찾아낼 수 있음!
print("쿠키 저장 완료")
driver.quit() # 10 , 나가기 까지 !!

수동으로 인증시작! in 30.sec
쿠키 저장 완료


In [6]:
#이제 프리패스로 뚫고가기!!!!!!
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By # 1, 뭘로 찾아올건지 선택하는거. (css, html 등)
from webdriver_manager.chrome import ChromeDriverManager
import pickle 
import time

options = Options()
options.add_argument("--start-maximized") # 3, 현재 우리 브라우저에 최적화된 사이즈로 크롤링하게끔하는 옵션
options.add_argument("window-size=1920x1080") 
options.add_argument("lang=ko_KR")

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.instagram.com/")

with open("instagram_cookies.pkl","rb") as f : # 2,찾아온 이름값을 읽어와야함 rb
    cookies = pickle.load(f) # 3, f의 실행값을 읽어와라. cookies는 여러개의 직렬값으로 들어왔기 때문에 리스트
for cookie in cookies:
    driver.add_cookie(cookie) # 4, 인증받은 사람인것처럼하는 크롤러
# 5, 쿠키값을 가진 드라이버가 새로고침한 상태로 접속! -> 로그인이 완료된 상태로 들어감
driver.refresh() #6, 바로 들어가면 사람이 아닌것같기 때문에 텀을 둠!
time.sleep(5)

# 7, 어디서 에러가 발생했는가
try :
    not_now = driver.find_element(By.XPATH,"//button[text()='나중에 하기']") # 8, 팝업이 뜨면 버튼의 역할을 하는 요소에 '나중에하기' 가 있으면 눌러라
    not_now.click()
    time.sleep(2)
except :
    pass # 9, 위 팝업이 뜨지 않으면 그냥 지나가라!

keyword = "무신사" # 10, 원하는 키워드를 넣으면 됨!
driver.get(f"https://www.instagram.com/explore/tags/{keyword}") 
time.sleep(5)

# 11, 특정 카테고리 중 어떤 컨텐츠는 도배를 하는경우 있음 도배 안뜨게 set
post_links = set() # 중복값을 받을 수 없게 만듦!
# 로딩스피너가 돌고나서 스크롤 내리기3번정도 반복할것!
for _ in range(1,4) :
    links = driver.find_elements(By.TAG_NAME, "a") # 12, 3번 돌동안 앵커태그 다 찾아옴
    for link in links :
        href = link.get_attribute("href") # 하이퍼래퍼런스에 매칭되어지는 값을 가져옴
        if href and "/p/" in href : # 14, 하이퍼레퍼런스가 있고, 축약되어진 문자에는 /p/가 다 있음!
            post_links.add(href)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 15, 높이 다시 측정, ; => 종료 시퀀스
    time.sleep(3)

results = []
#리스트 형태로 확실하게 바꾸기 -> 각각의 썸네일로 들어간 사이트 값을 가져오기 위해
for url in list(post_links) [:10] : # 16, 10개씩만 찾아와야안들킴
    driver.get(url)
    time.sleep(3)
    #각각의 컨텐츠에 있는 문안을 찾아오기
    caption = "" # 17, 캡션은 빈 문자열로!
    
    try :
        img_tag = driver.find_element(By.XPATH, "//img[@alt]") # 18, 각 요소들의 이미지 찾아오기
        caption = img_tag.get_attribute("alt").strip() # 19, alt속성을 찾아와서 여백을 없애줘
        results.append({"url":url, "caption": caption})# 캡션값을 찾아왔다면
    except Exception as e : ###########
        print(f"캡션 추출 실패 : {url} | 에러 : {e}") # 20, 몇번째 앵커에서 실패했는지 알아야함
        continue # 21, 다음 단계로 이동!
        # result 딕셔너리 값 추출하기
driver.quit()

for r in results :
    print(r["url"])
    print(r["caption"])
    print("="*60) # 구분선 만들어주기
    

https://www.instagram.com/p/DIbHP3ZyHtp/
musinsa.official님의 프로필 사진
https://www.instagram.com/p/DMjcsvkTJq2/
fashionsangsa님의 프로필 사진
https://www.instagram.com/p/DK9YPM3BEAH/
jae_jjang님의 프로필 사진
https://www.instagram.com/p/DIgHAdGSgLr/
Photo by 민수픽 on April 16, 2025. 사진 설명이 없습니다..
https://www.instagram.com/p/DMb5Sthp2Ht/
Photo shared by FASHIONBIZ 패션비즈 on July 22, 2025 tagging @the_hyundai, and @lotteshopping. 문구: 'MUSINSA e LOTTE DEPARTMENT STORE THE HYUNDAI 무신사 이어 롯데 현대백화점도 중고 거래 진출, 왜?'의 이미지일 수 있음.
https://www.instagram.com/p/DMDZkA7xenU/
Photo shared by 집 매거진 on July 13, 2025 tagging @naked_surf_club, and @jumiso_official. 사람 1명, 머리카락, 해변 및 문구: '여름이었다.. selter today' today'sphoto S photo 바다에서 이렇게 찍어줘! 1함께 함께 여름을 즐겨보아요 愛'의 이미지일 수 있음.
https://www.instagram.com/p/DK8jwsLPKYv/
Photo by 유키 on June 15, 2025. 사람 2명 및 문구: '이번 블프 지갑 탈탈 털릴 예정 무신사 블프 뭐싸지? 위시리스트 모음'의 이미지일 수 있음.
https://www.instagram.com/p/DMest0Py5Lw/
Photo by MODE magazine | 패션 매거진 모드 on July 23, 2025. 사람 6명 및 문구: '춘처 pinter

In [8]:
#이제 프리패스로 뚫고가기!!!!!!
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By # 1, 뭘로 찾아올건지 선택하는거. (css, html 등)
from webdriver_manager.chrome import ChromeDriverManager
import pickle 
import time

options = Options()
options.add_argument("--start-maximized") # 3, 현재 우리 브라우저에 최적화된 사이즈로 크롤링하게끔하는 옵션
options.add_argument("window-size=1920x1080") 
options.add_argument("lang=ko_KR")

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.instagram.com/")

with open("instagram_cookies.pkl","rb") as f : # 2,찾아온 이름값을 읽어와야함 rb
    cookies = pickle.load(f) # 3, f의 실행값을 읽어와라. cookies는 여러개의 직렬값으로 들어왔기 때문에 리스트
for cookie in cookies:
    driver.add_cookie(cookie) # 4, 인증받은 사람인것처럼하는 크롤러
# 5, 쿠키값을 가진 드라이버가 새로고침한 상태로 접속! -> 로그인이 완료된 상태로 들어감
driver.refresh() #6, 바로 들어가면 사람이 아닌것같기 때문에 텀을 둠!
time.sleep(5)

# 7, 어디서 에러가 발생했는가
try :
    not_now = driver.find_element(By.XPATH,"//button[text()='나중에 하기']") # 8, 팝업이 뜨면 버튼의 역할을 하는 요소에 '나중에하기' 가 있으면 눌러라
    not_now.click()
    time.sleep(2)
except :
    pass # 9, 위 팝업이 뜨지 않으면 그냥 지나가라!

keyword = "잠실" # 10, 원하는 키워드를 넣으면 됨!
driver.get(f"https://www.instagram.com/explore/tags/{keyword}") 
time.sleep(5)

# 11, 특정 카테고리 중 어떤 컨텐츠는 도배를 하는경우 있음 도배 안뜨게 set
post_links = set() # 중복값을 받을 수 없게 만듦!
# 로딩스피너가 돌고나서 스크롤 내리기3번정도 반복할것!
for _ in range(1,4) :
    links = driver.find_elements(By.TAG_NAME, "a") # 12, 3번 돌동안 앵커태그 다 찾아옴
    for link in links :
        href = link.get_attribute("href") # 하이퍼래퍼런스에 매칭되어지는 값을 가져옴
        if href and "/p/" in href : # 14, 하이퍼레퍼런스가 있고, 축약되어진 문자에는 /p/가 다 있음!
            post_links.add(href)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 15, 높이 다시 측정, ; => 종료 시퀀스
    time.sleep(3)

results = []
#리스트 형태로 확실하게 바꾸기 -> 각각의 썸네일로 들어간 사이트 값을 가져오기 위해
for url in list(post_links) [:10] : # 16, 10개씩만 찾아와야안들킴
    driver.get(url)
    time.sleep(3)
    #각각의 컨텐츠에 있는 문안을 찾아오기
    caption = "" # 17, 캡션은 빈 문자열로!
    
    try :
        img_tag = driver.find_element(By.XPATH, "//img[@alt]") # 18, 각 요소들의 이미지 찾아오기
        caption = img_tag.get_attribute("alt").strip() # 19, alt속성을 찾아와서 여백을 없애줘
        results.append({"url":url, "caption": caption})# 캡션값을 찾아왔다면
    except Exception as e : ###########
        print(f"캡션 추출 실패 : {url} | 에러 : {e}") # 20, 몇번째 앵커에서 실패했는지 알아야함
        continue # 21, 다음 단계로 이동!
        # result 딕셔너리 값 추출하기
driver.quit()

for r in results :
    print(r["url"])
    print(r["caption"])
    print("="*60) # 구분선 만들어주기
    

https://www.instagram.com/p/DA7pPzjIW4B/
Photo by 먹킷리스트 on October 09, 2024. 사진 설명이 없습니다..
https://www.instagram.com/p/DLZEqGIPdBM/
Photo by 서울핫플 공식계정 / 서울맛집 서울카페 서울여행 on June 26, 2025. 사람 1명, 포스터, 아지랑이 꽃 및 문구: '유료 유료광고포함 잠실 엘리자베스 랭그리터 블'엘리자베스랭그리터잔 전' 송파구 구잠실로209 로209 피헬앰배서더호헬 3층 뮤지엄209 ~9월28일'의 만화일 수 있음.
https://www.instagram.com/p/DMaNTMJyyNY/
Photo by 먹퀘스트 [맛집 •카페 • 신상리뷰] on July 22, 2025. 타워, 경복궁 및 문구: 'SEOUL 송리단길 세시의 데이트 피기구산!'의 이미지일 수 있음.
https://www.instagram.com/p/DMm4mdovyIY/
Photo shared by 무데이트 on July 27, 2025 tagging @alley_nine. 두부, 야채 및 문구: '스페인에 가지아고 이고 吉 가지 양송이 타파스를 수 이다? 잠실 잠실엘로 엘리나인 @moodate_'의 이미지일 수 있음.
https://www.instagram.com/p/DIi39wHTYIm/
Photo by 핫츄 | Travel Designer✈️🎨 on April 17, 2025. 사진 설명이 없습니다..
https://www.instagram.com/p/DCiUnlcTGBy/
Photo by 빵모닝 on November 18, 2024. 사진 설명이 없습니다..
https://www.instagram.com/p/DLSbhHuyfnt/
Photo by 마로롱ㅣ서울맛집,핫플,신상,팝업₍⑅ᐢ. ̬.ᐢ₎ෆ on June 24, 2025. 장난감 및 문구: '잠실에 잠실에이런곳이!? 이런곳이!? 신상 신상키티천국소품샵 계접강정지협를 7, 키티천국 소품샵 계전국